In [8]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder

In [4]:
train_dec = pd.read_csv('../../Data/train_before_dec_v1.csv', delimiter='|')

In [32]:
y_train = train_dec['week']
x_train = train_dec.drop(columns=['week'])
# transformer = make_column_transformer(
#     (OneHotEncoder(), ['userID', 'itemID']),
#     remainder='passthrough')
# transformed= transformer.fit_transform(x_train)
# transformed_df = pd.DataFrame(transformed)
# x_train = transformed_df

In [6]:
test_dec = pd.read_csv('../../Data/test_dec.csv', delimiter='|')

In [33]:
model = LogisticRegression()
model.fit(x_train, y_train)

In [ ]:
test_x = pd.read_csv('../../Data/submission_dec.csv', delimiter='|')
test_x = test_x.drop(columns=['prediction'])
test_x['month'] = 12
cols = ['month', 'userID', 'itemID']
test_x = test_x.reindex(columns=cols)
# test_x = pd.DataFrame(transformer.transform(test_x))

In [ ]:
pred_y = model.predict(test_x)

In [24]:
def count_points(pred, gold):
    df = pd.merge(pred, gold, on=['userID', 'itemID'], suffixes=('_pred', '_gold'))
    df['points'] = df.apply(_compute_points_for_row, axis=1)
    return df['points'].sum()

def _compute_points_for_row(row):
    y_pred, y_gold = row.prediction_pred, row.prediction_gold
    if y_pred == y_gold:
        # one point if "no order" (0) is predicted correctly; three points if order week is predicted correctly
        return 1 if y_pred == 0 else 3
    # one point if order is predicted correctly (but not the correct week), otherwise zero points
    return 1 if (y_pred > 0 and y_gold > 0) else 0

In [ ]:
test_x = pd.read_csv('../../Data/submission_dec.csv', delimiter='|')
test_x = test_x.drop(columns=['prediction'])
test_x['month'] = 12
cols = ['month', 'userID', 'itemID']
test_x = test_x.reindex(columns=cols)
pred = test_x.copy()
pred['prediction'] = pred_y

In [ ]:
points = count_points(pred, test_dec)
max_points = count_points(test_dec, test_dec)
score = points / max_points

In [ ]:
score

0.34655701627381713

In [ ]:
0.34655701627381713